#### 스타벅스 부산시 매장위치 크롤링

##### 스타벅스 웹사이트
- https://www.starbucks.co.kr/index.do 

##### 셀레니움 등 라이브러 임포트

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.autonotebook import tqdm

C:\Users\Administrator\AppData\Local\Temp\ipykernel_10396\3272571517.py:5: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
# 크롬 웹드라이버 실행
driver = webdriver.Chrome()
url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
driver.get(url)

In [3]:
# 웹드라이버 선택할 위치값 작성(20년 8월과 동일)
btn_busan = '#container > div > form > fieldset > div > section > article.find_store_cont > ' + \
            'article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > ' + \
            'li:nth-child(6) > a'
driver.find_element(By.CSS_SELECTOR, btn_busan).click()

In [4]:
# 부산 선택된 상태
# '전체' 클릭
btn_all = '#mCSB_2_container > ul > li > a' # 첫번째 값이기 때문에 nth-child 도 필요없음
driver.find_element(By.CSS_SELECTOR, btn_all).click()

In [5]:
# 스타벅스는 한화면에 전부 리스트업되기때문에 함수를 만들필요도 없음
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
soup

In [6]:
# 검색결과 위치 
starbuks_soup_list = soup.select('li.quickResultLstCon')
len(starbuks_soup_list)

137

In [7]:
starbuks_soup_list[0]

<li class="quickResultLstCon" data-code="3705" data-hlytag="null" data-index="0" data-lat="35.262466" data-long="129.091353" data-name="금정남산동DT" data-storecd="1605" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="금정남산동DT" data-store="1605" data-yn="N">금정남산동DT  </strong> <p class="result_details">부산광역시 금정구 중앙대로 1989 (남산동)<br/>1522-3232</p> <i class="pin_generalDT">리저브 매장 2번</i></li>

In [8]:
# 매장정보 확인
starbuks_store = starbuks_soup_list[0]
name = starbuks_store.select('strong')[0].text.strip()
lat = starbuks_store['data-lat'].strip()
lng = starbuks_store['data-long'].strip()
store_type = starbuks_store.select('i')[0]['class'][0].split('_')[1] # _로 자르면 두번째값이 general/reserve
address = starbuks_store.select('p')[0].get_text(separator='|').split('|')[0]

print(name) # 매장명
print(lat, lng, sep=' / ') # 매장 지도 위치
print(store_type)
print(address)

금정남산동DT
35.262466 / 129.091353
generalDT
부산광역시 금정구 중앙대로 1989 (남산동)


In [9]:
# 137개 매장 데이터 가져오기
starbuks_busan_list = []

for item in tqdm(starbuks_soup_list):
    name = item.select('strong')[0].text.strip()
    lat = item['data-lat'].strip()
    lng = item['data-long'].strip()
    store_type = item.select('i')[0]['class'][0].split('_')[1] # _로 자르면 두번째값이 general/reserve
    address = item.select('p')[0].get_text(separator='|').split('|')[0]

    starbuks_busan_list.append([name, lat, lng, store_type, address])

100%|██████████| 137/137 [00:00<00:00, 5430.67it/s]


In [10]:
len(starbuks_busan_list)

137

In [11]:
# DataFrame 생성
columns = ['매장명', '위도', '경도', '매장타입', '주소']
df = pd.DataFrame(starbuks_busan_list, columns=columns)
df.tail()

,매장명,위도,경도,매장타입,주소
132,해운대 엑스더스카이,35.1599648,129.1697564,general,부산광역시 해운대구 달맞이길30
133,센텀신세계3F,35.16906348,129.12906568,general,부산광역시 해운대구 센텀남대로 35 (우동)
134,해운대중동역,35.1671846,129.1678855,general,부산광역시 해운대구 좌동순환로 6 (중동)
135,센텀몰1F,35.16981023,129.12778297,general,부산광역시 해운대구 센텀4로 15 (우동)
136,그랜드조선 부산,35.1600338,129.1630512,general,부산광역시 해운대구 해운대해변로 292 (중동)


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   매장명     137 non-null    object
 1   위도      137 non-null    object
 2   경도      137 non-null    object
 3   매장타입    137 non-null    object
 4   주소      137 non-null    object
dtypes: object(5)
memory usage: 5.5+ KB


In [13]:
# 엑셀로 저장
df.to_excel('./busan_starbuks.xlsx', index=False)